In [99]:
from pygraphblas import *
from pygraphblas import lib
from pygraphblas.types import Type, binop
#from pygraphblas.gviz import draw, draw_vis, draw_vector, draw_matrix
import numpy as np
import pprint
from pyvis import network as net

import this
options_set(nthreads=12)
DECIMALS = 9

In [100]:


class GS(Type):

    _base_name = "UDT"
    _numpy_t = None
    members = ['double w', 'int64_t n']
    one = (np.float64('inf'), np.int64(0))
    # equal operation is not working with "iseq" operation, D.iseq(D1)
    @binop(boolean=True)
    def EQ(z, x, y):
        if np.round(x.w, DECIMALS) == np.round(y.w, DECIMALS) and x.n == y.n:
            z = True
        else:
            z = False

    @binop()
    def PLUS(z, x, y):
        if np.round(x.w, DECIMALS) < np.round(y.w, DECIMALS):
            z.w = x.w
            z.n = x.n
        elif np.round(x.w, DECIMALS) == np.round(y.w, DECIMALS):
            z.w = x.w
            z.n = x.n + y.n
        else:
            z.w = y.w
            z.n = y.n

    @binop()
    def TIMES(z, x, y):
        z.w = x.w + y.w
        z.n = x.n * y.n

GS_monoid = GS.new_monoid(GS.PLUS, GS.one)
GS_semiring = GS.new_semiring(GS_monoid, GS.TIMES)

def shortest_path(matrix, start):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    v = matrix.extract_row(start)
    v_k = matrix.extract_row(start)
    with GS_semiring:
        for _ in range(matrix.nrows+1):
            v_k = v_k @ matrix
            v = v + v_k
    return v

A = Matrix.sparse(GS, 6, 6)
A[0,1] = (9.0, 1)
A[0,3] = (3.0, 1)
A[0,5] = (4.0, 1)
A[1,2] = (8.0, 1)
A[3,4] = (6.0, 1)
A[3,5] = (1.0, 1)
A[4,2] = (4.0, 1)
A[1,5] = (7.0, 1)
A[5,4] = (2.0, 1)

N = net.Network(notebook=True, directed=True, cdn_resources='in_line')
for i, j, v in A:
    N.add_node(i, label=str(i), shape='circle')
    N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v[0]))
N.show('graph.html')

graph.html


In [101]:
# g = draw(A, label_vector=shortest_path(A, 0))
def shortest_path_FW(matrix):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    D_k = matrix.dup()
    D = matrix.dup()
    with GS_semiring:
        for _ in range(matrix.nrows):
            D_k @= matrix
            D += D_k
    return D

In [102]:
def shortest_path_FW_early_stop(matrix):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    D_k = matrix.dup()
    D = matrix.dup()
    D_prev = matrix.dup()
    with GS_semiring:
        while True:
            D_prev = D.dup()
            D_k @= matrix
            D += D_k
            if D.iseq(D_prev):
                break
    return D

In [103]:
D = shortest_path_FW(A)

In [104]:
for i, j, k in D:
    print(i, j, D[i, j])
# i, j, shortest path distance, number of shortest paths

0 1 (9.0, 1)
0 2 (10.0, 2)
0 3 (3.0, 1)
0 4 (6.0, 2)
0 5 (4.0, 2)
1 2 (8.0, 1)
1 4 (9.0, 1)
1 5 (7.0, 1)
3 2 (7.0, 1)
3 4 (3.0, 1)
3 5 (1.0, 1)
4 2 (4.0, 1)
5 2 (6.0, 1)
5 4 (2.0, 1)


In [105]:
D1 = shortest_path_FW_early_stop(A)
for i, j, k in D1:
    print(i, j, D1[i, j])
with GS_semiring:
    print(D.iseq(D1))

0 1 (9.0, 1)
0 2 (10.0, 1)
0 3 (3.0, 1)
0 4 (6.0, 2)
0 5 (4.0, 2)
1 2 (8.0, 1)
1 4 (9.0, 1)
1 5 (7.0, 1)
3 2 (7.0, 1)
3 4 (3.0, 1)
3 5 (1.0, 1)
4 2 (4.0, 1)
5 2 (6.0, 1)
5 4 (2.0, 1)
True


In [106]:
N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
labels = {}
# i, j, shortest path distance, number of shortest paths
for i, j, v in D:
    if j not in labels:
        
        labels[j] = [f"({i},{j},{v[0]},{v[1]})"]
    else:
        labels[j].append(f"({i},{j},{v[0]},{v[1]})")
for i, j, v in A:
    if i in labels:
        N.add_node(i, label='\n'.join(labels[i]), shape='circle')
    else:
        N.add_node(i, label=str(i), shape='circle')
    if j in labels:
        N.add_node(j, label='\n'.join(labels[j]), shape='circle')
    else:
        N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v))     
N.show('graph_shortest_paths.html')

graph_shortest_paths.html


In [107]:
def algebraic_IPC(Adj, x):
    global DECIMALS
    x = x.astype(np.float64)
    n = Adj.shape[0]
    A = Matrix.sparse(GS, n, n)
    w_s_r = np.float64(0)
    w_v = np.zeros(n, dtype=np.float64)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    for k, v in Adj.items():
        i = k[0]
        j = k[1]
        if i == j:
            A[i,j] = (np.float64('inf'), np.int64(0))
        else:
            A[i,j] = (float(v), np.int64(1))
    D = shortest_path_FW(A)
    #C = Vector.sparse(FP64, n)  
    C = np.zeros(n, dtype=np.float64)
    for v in range(n):
        for s in D.extract_col(v).indices:
            for r in D.extract_row(v).indices:
                if x[r] - x[s] <= 0:
                    continue
                elif s == v or r == v or s == r:
                    continue
                elif np.round(D[s, r][0], DECIMALS) == np.round(D[s, v][0] + D[v, r][0], DECIMALS):
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v]) 
                    C[v] += np.float64(D[s, v][1] * D[v, r][1]) / np.float64(D[s, r][1]) * w_v_sr
    return C

In [108]:
from scipy.sparse import dok_matrix

Adj = dok_matrix((6, 6), dtype=float) 
Adj[0,1] = 9.0
Adj[0,3] = 3.0
Adj[0,5] = 4.0
Adj[1,2] = 8.0
Adj[3,4] = 6.0
Adj[3,5] = 1.0
Adj[4,2] = 4.0
Adj[1,5] = 7.0
Adj[5,4] = 2.0

states = np.array([0, 0, 0.4, 0, 0, 0])
IPC = algebraic_IPC(Adj, states)
print(IPC)

[0.    0.    0.    0.125 0.75  0.5  ]


In [109]:
def algebraic_IPC_with_paths(Adj, x):
    x = x.astype(np.float64)
    n = Adj.shape[0]
    A = Matrix.sparse(GS, n, n)
    A1 = Matrix.sparse(FP64, n, n)
    w_s_r = np.float64(0)
    w_v = np.zeros(n, dtype=np.float64)
    # compute relative contribution coefficients (weights)
    for s in range(n):
        for r in range(n):
            if x[r] - x[s] > 0:
                w_s_r += x[r] - x[s]
                w_v[s] += x[r] - x[s]
                w_v[r] += x[r] - x[s]
    for k, v in Adj.items():
        i = k[0]
        j = k[1]
        if i == j:
            A[i, j] = (np.float64('inf'), np.int64(0))
        else:
            A[i, j] = (v, np.int64(1))
            A1[i,j] = np.float64(v)
    D = shortest_path_FW(A)
    D1 = Matrix.sparse(FP64, n, n)
    for i, j, v in D:
        D1[i,j] = v[0]
    print(A1)
    print(D1)
    pred = dict()
    with FP64.plus:
        for i in range(n):
            pred[i] = dict()
            d = D1.extract_row(i)
            d[i] = 0
            for j in d.indices:
                col_j = A1.extract_col(j)
                col_j.emult(d, mult_op=FP64.plus, out=col_j)
                #pred_j = list((col_j == d[j]).indices)
                #if pred_j:
                pred[i][j] = []
                for v in col_j.indices:
                    if np.round(col_j[v], DECIMALS) == np.round(d[j], DECIMALS):
                        pred[i][j].append(v)
    C = np.zeros(n, dtype=np.float64)
    paths = dict()
    v_paths = dict()
    for v in range(n):
        for s in D.extract_col(v).indices:
            for r in D.extract_row(v).indices:
                if x[r] - x[s] <= 0:
                    continue
                elif s == v or r == v or s == r:
                    continue
                elif np.round(D[s, r][0], DECIMALS) == np.round(D[s, v][0] + D[v, r][0], DECIMALS):
                    w_v_sr = (x[r] - x[s]) / (w_s_r - w_v[v]) 
                    C[v] += np.float64(D[s, v][1] * D[v, r][1]) / np.float64(D[s, r][1]) * w_v_sr
                    if (s, r) not in paths:
                        paths[(s, r)] = st_paths(pred[s], s, r)
                    if v not in v_paths:
                        v_paths[v] = set()
                    #for p in paths[(s, r)]:
                    #    if v in p:
                    #        v_paths[v].append(p)
                    sv_paths = st_paths(pred[s], s, v)
                    vr_paths = st_paths(pred[s], v, r)
                    for p1 in sv_paths:
                        for p2 in vr_paths:
                            v_paths[v].add(tuple(p1 + p2[1:]))
    return C, v_paths, paths, pred


def st_paths(pred, s, t):
    stack = [[t, 0]]
    top = 0
    paths = []
    while top >= 0:
        node, i = stack[top]
        if node == s:
            paths.append([p for p, n in reversed(stack[:top+1])])
        if node not in pred:
            continue
        if len(pred[node]) > i:
            top += 1
            if top == len(stack):
                stack.append([pred[node][i],0])
            else:
                stack[top] = [pred[node][i],0]
        else:
            stack[top-1][1] += 1
            top -= 1
    return paths

In [110]:
C, v_paths, paths, pred = algebraic_IPC_with_paths(Adj, states)
print('Centrality')
pprint.pprint(C)
print('Predecessors')
pprint.pprint(pred)
print('Shortest Inverse Percolation Paths through v')
pprint.pprint(v_paths)
print('Shortest Inverse Percolation Paths')
pprint.pprint(paths)

      0  1  2  3  4  5
  0|   9.0   3.0   4.0|  0
  1|      8.0      7.0|  1
  2|                  |  2
  3|            6.01.0|  3
  4|      4.0         |  4
  5|            2.0   |  5
      0  1  2  3  4  5
      0  1  2  3  4  5
  0|   9.010.03.06.04.0|  0
  1|      8.0   9.07.0|  1
  2|                  |  2
  3|      7.0   3.01.0|  3
  4|      4.0         |  4
  5|      6.0   2.0   |  5
      0  1  2  3  4  5
Centrality
array([0.   , 0.   , 0.   , 0.125, 0.75 , 0.5  ])
Predecessors
{0: {0: [], 1: [0], 2: [4], 3: [0], 4: [5], 5: [0, 3]},
 1: {1: [], 2: [1], 4: [5], 5: [1]},
 2: {2: []},
 3: {2: [4], 3: [], 4: [5], 5: [3]},
 4: {2: [4], 4: []},
 5: {2: [4], 4: [5], 5: []}}
Shortest Inverse Percolation Paths through v
{3: {(0, 3, 5, 4, 2)},
 4: {(5, 4, 2), (3, 5, 4, 2), (0, 5, 4, 2), (0, 3, 5, 4, 2)},
 5: {(3, 5, 4, 2), (0, 5, 4, 2), (0, 3, 5, 4, 2)}}
Shortest Inverse Percolation Paths
{(0, 2): [[0, 5, 4, 2], [0, 3, 5, 4, 2]],
 (3, 2): [[3, 5, 4, 2]],
 (5, 2): [[5, 4, 2]]}


In [111]:
# Shortest path tree rooted at 0.
s = 0
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_0.html')


sssp_s_0.html


In [112]:
# Shortest path tree rooted at 1.
s = 1
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_1.html')

sssp_s_1.html


In [113]:
# Shortest path tree rooted at 2.
s = 2
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_2.html')

sssp_s_2.html


In [114]:
# Shortest path tree rooted at 3.
s = 3
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_3.html')

sssp_s_3.html


In [115]:
# Shortest path tree rooted at 4.
s = 4
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_4.html')

sssp_s_4.html


In [116]:
# Shortest path tree rooted at 5.
s = 5
if s in pred:
    N = net.Network(notebook=True, directed=True, height=300, cdn_resources='in_line')
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[s]:
            continue
        if i in pred[s][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp_s_5.html')

sssp_s_5.html
